In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Load data
data = pd.read_csv("master.csv")  # Replace with your dataset

# Normalize data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Convert data to PyTorch tensors
data_tensor = torch.tensor(data_scaled, dtype=torch.float32)

# Define the Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize the model
input_dim = data_tensor.shape[1]
encoding_dim = 10  # Adjust based on how much you want to compress
model = Autoencoder(input_dim, encoding_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
batch_size = 32
data_loader = torch.utils.data.DataLoader(data_tensor, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch in data_loader:
        # Forward pass
        outputs = model(batch)
        loss = criterion(outputs, batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Extract compressed player representations
with torch.no_grad():
    compressed_data = model.encoder(data_tensor).numpy()

# Save compressed data to a DataFrame
compressed_df = pd.DataFrame(compressed_data)
compressed_df.to_csv("compressed_data.csv", index=False)

print("Compressed data saved to compressed_data.csv")
